In [1]:
import torch

## Activation Functions: Sigmoid, ReLU

In [2]:
class Activation_Sigmoid:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = 1 / (1 + torch.exp(-inputs))

    def backward(self, dvalues):
        sigmoid_derivative = self.output * (1 - self.output)
        self.dinputs = dvalues * sigmoid_derivative

class Activation_ReLU:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = torch.maximum(torch.tensor(0), inputs)

    def backward(self, dvalues):
        self.dinputs = dvalues.clone()
        self.dinputs[self.inputs <= 0] = 0

## Dense Layer

In [3]:
class DenseLayer:
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01 * torch.rand(n_inputs, n_neurons)
        self.biases = torch.zeros((1, n_neurons))

    def forward(self, inputs):
        self.inputs = inputs
        self.output = torch.matmul(inputs, self.weights) + self.biases

    def backward(self, dvalues):
        self.dweights = torch.matmul(self.inputs.T, dvalues)
        self.dbiases = torch.sum(dvalues, axis=0, keepdims=True)
        self.dinputs = torch.matmul(dvalues, self.weights.T)

## Loss: Mean Squared Error

In [4]:
class Loss_MSE:
    def forward(self, y_pred, y_true):
        return torch.mean((y_pred - y_true) ** 2)

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        outputs = len(dvalues[0])
        self.dinputs = 2 * (dvalues - y_true) / outputs
        self.dinputs = self.dinputs / samples


# Neural Network Architecture

## Input Layer
- Features: 2

## Hidden Layer
- Neurons: 2
- Activation Function: ReLU

## Output Layer
- Neurons: 2
- Activation Function: Sigmoid

## Loss function
- Loss Function: Mean Squared Error (MSE)


## Sample Usage

In [5]:
learning_rate = 0.1
epochs = 1000  # Number of iterations for training

# Instantiate the layers, activations, and loss function
layer1 = DenseLayer(2, 2)  # 2 inputs, 2 neurons in the hidden layer
activation1 = Activation_ReLU()

layer2 = DenseLayer(2, 2)  # 2 neurons from the hidden layer to 2 output neurons
activation2 = Activation_Sigmoid()

loss_function = Loss_MSE()

X = torch.tensor([[1.0, 2.0], [0.5, -1.0]])  # Input data
Y = torch.tensor([[1.0, 0.0], [0.0, 1.0]])  # True labels

# Initial forward pass for initial loss
layer1.forward(X)
activation1.forward(layer1.output)

layer2.forward(activation1.output)
activation2.forward(layer2.output)

# Initial loss calculation
initial_loss = loss_function.forward(activation2.output, Y)
print(f'Initial Loss: {initial_loss}')

# Training loop
for epoch in range(epochs):
    # Forward pass
    layer1.forward(X)
    activation1.forward(layer1.output)

    layer2.forward(activation1.output)
    activation2.forward(layer2.output)

    # Loss calculation
    loss = loss_function.forward(activation2.output, Y)

    # Backward pass
    loss_function.backward(activation2.output, Y)
    activation2.backward(loss_function.dinputs)

    layer2.backward(activation2.dinputs)
    activation1.backward(layer2.dinputs)

    layer1.backward(activation1.dinputs)

    # Update weights and biases
    layer1.weights -= learning_rate * layer1.dweights
    layer1.biases -= learning_rate * layer1.dbiases

    layer2.weights -= learning_rate * layer2.dweights
    layer2.biases -= learning_rate * layer2.dbiases

    # Print loss every 100 epochs
    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss}')


Initial Loss: 0.2500019073486328
Epoch 0, Loss: 0.2500019073486328
Epoch 100, Loss: 0.24955417215824127
Epoch 200, Loss: 0.22268623113632202
Epoch 300, Loss: 0.10445977747440338
Epoch 400, Loss: 0.06082547456026077
Epoch 500, Loss: 0.03991616144776344
Epoch 600, Loss: 0.028526008129119873
Epoch 700, Loss: 0.021698269993066788
Epoch 800, Loss: 0.017273079603910446
Epoch 900, Loss: 0.014223725534975529
